# Load Data

In [3]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import datetime as dt
from datetime import date
import pickle

#declare some variables
today = pd.Timestamp.today()
path_in = 'data/data_lokad'
path_out ='data/temp01'

## TODO 
# import reworks?

## Purchasing data

In [21]:
#load current file + format + add data source column
PURCHASING = pd.read_csv(path_in + '/LOKAD_PURCHASING.txt',delimiter='|',
                           dtype={
                               'PN':'str',
                               'PNM':'str',
                           })
PURCHASING['PURDATE'] = pd.to_datetime(PURCHASING['PURDATE'],format='%Y-%m-%d',errors='coerce')
PURCHASING['RECDATE'] = pd.to_datetime(PURCHASING['RECDATE'],format='%Y-%m-%d',errors='coerce')
PURCHASING['DATA_SOURCE'] = 'SQL'
#load Histo PO and format
HISTO_PO_ARO = pd.read_csv(path_in + '/HISTORICAL_PURCHASES_ARO.txt',delimiter='|',
                            dtype={'PNM':'str','item':'str'},
                            usecols=[1,2,3,4,5,6,7,8,9,10,11])
HISTO_PO_ARO = HISTO_PO_ARO.loc[HISTO_PO_ARO['QTYPENDING'] == 0] #delete suspicious open POs
HISTO_PO_ARO['DEPT_NAME'] = 'PURCHASING ARO'
HISTO_PO_ARO['DATA_SOURCE'] = 'HISTO ARO'
HISTO_PO_ARO['PURDATE'] = pd.to_datetime(HISTO_PO_ARO['PURDATE'], format='%m/%d/%Y %H:%M:%S')
HISTO_PO_ARO['RECDATE'] = pd.to_datetime(HISTO_PO_ARO['RECDATE'], format='%m/%d/%Y %H:%M:%S')

HISTO_PO_BIC = pd.read_csv(path_in + '/HISTORICAL_PURCHASES_BIC.txt',delimiter='|',
                          dtype={'PNM':'str'},
                          usecols=[1,2,3,4,5,6,7,8,9,10,11])
HISTO_PO_BIC = HISTO_PO_BIC.loc[HISTO_PO_BIC['QTYPENDING'] == 0] #delete suspicious open POs
HISTO_PO_BIC['DEPT_NAME'] = 'PURCHASING BIC'
HISTO_PO_BIC['DATA_SOURCE'] = 'HISTO BIC'
HISTO_PO_BIC['PURDATE'] = pd.to_datetime(HISTO_PO_BIC['PURDATE'], format='%m/%d/%Y %H:%M:%S')
HISTO_PO_BIC['RECDATE'] = pd.to_datetime(HISTO_PO_BIC['RECDATE'], format='%m/%d/%Y %H:%M:%S')

#append histo data
PURCHASING = PURCHASING.append(HISTO_PO_ARO, ignore_index = True, sort = False)
PURCHASING = PURCHASING.append(HISTO_PO_BIC, ignore_index = True, sort = False)

#cleanining
PURCHASING['QTYRECEIVED'] = pd.to_numeric(PURCHASING['QTYRECEIVED'], downcast='integer')
PURCHASING['QTYPENDING'] = pd.to_numeric(PURCHASING['QTYPENDING'], downcast='integer')
PURCHASING['EXT_COST'] = PURCHASING['COST'] * PURCHASING['QTYRECEIVED']
PURCHASING = PURCHASING[~PURCHASING['GEO_CODE'].isnull()] 
PURCHASING = PURCHASING[(PURCHASING['RECDATE'] <= today)|(PURCHASING['RECDATE']).isnull()] #remove POs received in the futur

PURCHASING.to_pickle(path_out+'/PURCHASING.pkl')

## Stock Detail data

In [66]:
STOCK = pd.read_csv(path_in + '/LOKAD_STOCK_DETAIL.txt',delimiter='|',
                           dtype={
                               'PN':'str',
                               'PNM':'str'
                           })
STOCK['RECDATE'] = pd.to_datetime(STOCK['RECDATE'], format='%Y-%m-%d') #what to do the null ones??
#STOCK['UNIT_COST'] = STOCK['UNIT_COST'].str.replace('$','') # change the negative costs ((WTF))
#STOCK['UNIT_COST'] = STOCK['UNIT_COST'].str.replace('(','-')
#STOCK['UNIT_COST'] = STOCK['UNIT_COST'].str.replace(')','')
STOCK['UNIT_COST'] = pd.to_numeric(STOCK['UNIT_COST'], downcast='float')


STOCK.to_pickle(path_out + '/STOCK.pkl')

## Parts Master data

In [81]:
PARTS_MASTER = pd.read_csv(path_in + '/LOKAD_PARTS_MASTER.txt',delimiter='|',
                            dtype={
                                'PN':'str',
                                'PNM':'str',
                            })

PARTS_MASTER['LIST_PRICE'] = PARTS_MASTER['LIST_PRICE'].str.replace('$','') # change the negative costs ((WTF))
PARTS_MASTER['LIST_PRICE'] = PARTS_MASTER['LIST_PRICE'].str.replace('(','-')
PARTS_MASTER['LIST_PRICE'] = PARTS_MASTER['LIST_PRICE'].str.replace(')','')
PARTS_MASTER['PRICEDATE'] = pd.to_datetime(PARTS_MASTER['PRICEDATE'], format='%Y-%m-%d', errors='coerce')
PARTS_MASTER['LIST_PRICE'] = pd.to_numeric(PARTS_MASTER['LIST_PRICE'], downcast='float')

PARTS_MASTER.to_pickle(path_out + '/PARTS_MASTER.pkl')

## Alternates data

In [2]:

ALTERNATES = pd.read_csv(path_in + '/LOKAD_ALTERNATES.txt',delimiter='|',
                            dtype={
                                'PNM':'str',
                                'ALT_PNM':'str',
                                'PN':'str',
                                'ALT_PN':'str'
                            })

ALTERNATES.to_pickle(path_out+'\ALTERNATES.pkl')

NameError: name 'pd' is not defined

## Sales Orders data

In [4]:
#load current file
SALES_ORDERS = pd.read_csv(path_in + '/LOKAD_SALES_ORDERS.txt',delimiter='|',
                            dtype={
                                'PNM':'str',
                                'PN':'str'
                            })

SALES_ORDERS['DATA_SOURCE'] = 'SQL'
SALES_ORDERS['ENTRY_DATE'] = pd.to_datetime(SALES_ORDERS['ENTRY_DATE'], format='%Y-%m-%d', errors='coerce')
SALES_ORDERS['SHIP_DATE'] = pd.to_datetime(SALES_ORDERS['SHIP_DATE'], format='%Y-%m-%d', errors='coerce')
SALES_ORDERS = SALES_ORDERS.drop(columns='ZNEG')

#load histo SO and format
HISTO_SO_BIC = pd.read_csv(path_in + '/HISTORICAL_SALES_BIC.txt',delimiter='|',
                          dtype={'PNM':'str'})
HISTO_SO_BIC = HISTO_SO_BIC.drop(columns='ID')
HISTO_SO_BIC['ENTRY_DATE'] = pd.to_datetime(HISTO_SO_BIC['ENTRY_DATE'], format = '%m/%d/%Y %H:%M:%S' )
HISTO_SO_BIC['SHIP_DATE'] = pd.to_datetime(HISTO_SO_BIC['ENTRY_DATE'], format = '%m/%d/%Y %H:%M:%S' )
HISTO_SO_BIC['DATA_SOURCE'] = 'HISTO BIC'

#parse histo data
SALES_ORDERS = SALES_ORDERS.append(HISTO_SO_BIC, ignore_index = True, sort = False)

# save as pickle
SALES_ORDERS.to_pickle(path_out + '/SALES_ORDERS.pkl')

#drop dataframe
del SALES_ORDERS
del HISTO_SO_BIC

## WO Bom data

In [118]:
#Load Current file
WO_BOM = pd.read_csv(path_in + '/LOKAD_WO_BOM.txt',delimiter='|',
                            dtype={
                                'PNM':'str',
                                'PN':'str'
                            })
WO_BOM['ADD_DATE'] = pd.to_datetime(WO_BOM['ADD_DATE'],format='%Y-%m-%d',errors='coerce')
WO_BOM['ISSUE_DATE'] = pd.to_datetime(WO_BOM['ISSUE_DATE'],format='%Y-%m-%d',errors='coerce')
WO_BOM['DATA_SOURCE'] = 'SQL'
WO_BOM['EXT_COST'] = WO_BOM['QTY_ISSUED'] * WO_BOM['COST']

#load histo WOB and format
HISTO_WOB_ARO = pd.read_csv(path_in + '/HISTORICAL_WO_BOM_ARO.txt',delimiter='|',
                           dtype={'PNM':'str',
                                  'WONO':'str'
                                  })
HISTO_WOB_ARO = HISTO_WOB_ARO.drop(columns=['ID'])

HISTO_WOB_ARO['ADD_DATE'] = pd.to_datetime(HISTO_WOB_ARO['ADD_DATE'],  format='%m/%d/%Y %H:%M:%S')
HISTO_WOB_ARO['ISSUE_DATE'] = pd.to_datetime(HISTO_WOB_ARO['ISSUE_DATE'],  format='%m/%d/%Y %H:%M:%S')
HISTO_WOB_ARO['DATA_SOURCE'] = 'HISTO ARO'
HISTO_WOB_ARO['GEO_CODE'] = 'DOR' #shouldn't been needed later

HISTO_WOB_BIC = pd.read_csv(path_in + '/HISTORICAL_WO_BOM_BIC.txt',delimiter='|',
                           dtype={'PNM':'str','WONO':'str'})
HISTO_WOB_BIC['ADD_DATE'] = pd.to_datetime(HISTO_WOB_BIC['ADD_DATE'],  format='%m/%d/%Y %H:%M:%S')
HISTO_WOB_BIC['ISSUE_DATE'] = pd.to_datetime(HISTO_WOB_BIC['ISSUE_DATE'],  format='%m/%d/%Y %H:%M:%S')
HISTO_WOB_BIC = HISTO_WOB_BIC.drop(columns=['ID'])
HISTO_WOB_BIC['DATA_SOURCE'] = 'HISTO BIC'
HISTO_WOB_BIC['GEO_CODE'] = 'MIA MRO' #shouldn't been needed later

#Append histo to current
WO_BOM= WO_BOM.append(HISTO_WOB_ARO, ignore_index = True, sort = False)
WO_BOM= WO_BOM.append(HISTO_WOB_BIC, ignore_index = True, sort = False)

#Cleaning
WO_BOM = WO_BOM[~WO_BOM['ACTIVITY'].str.contains('2014')] #bad conversions from histo data
WO_BOM = WO_BOM[~WO_BOM['ACTIVITY'].str.contains('2015')]
WO_BOM = WO_BOM[~WO_BOM['ACTIVITY'].str.contains('2016')]
WO_BOM = WO_BOM.loc[WO_BOM['ACTIVITY'] != 'CLOSED']  #bad conversions from histo data


#save as pickly
WO_BOM.to_pickle(path_out + '/WO_BPM.pkl')

## WO Header data

In [120]:
#Load current file
WO_HEADER = pd.read_csv(path_in + '/LOKAD_WO_HEADER.txt',delimiter='|',
                            dtype={
                                'PNM':'str',
                                'PN':'str'
                            })
WO_HEADER['DATA_SOURCE'] = 'SQL'
WO_HEADER['ENTRY_DATE'] = pd.to_datetime(WO_HEADER['ENTRY_DATE'],format='%Y-%m-%d',errors='coerce')
WO_HEADER['CLOSE_DATE'] = pd.to_datetime(WO_HEADER['CLOSE_DATE'],format='%Y-%m-%d',errors='coerce')

#load histo WOH and format
HISTO_WOH_ARO = pd.read_csv(path_in + '/HISTORICAL_WO_HEADER_ARO.txt',delimiter='|',
                           dtype={'PNM':'str','WONO':'str'},
                           usecols=[1,2,3,4,5,6,7,8,9,10,11,12])
HISTO_WOH_ARO['ENTRY_DATE'] = pd.to_datetime(HISTO_WOH_ARO['ENTRY_DATE'],  format='%m/%d/%Y %H:%M:%S')
HISTO_WOH_ARO['CLOSE_DATE'] = pd.to_datetime(HISTO_WOH_ARO['CLOSE_DATE'],  format='%m/%d/%Y %H:%M:%S')
HISTO_WOH_ARO['DATA_SOURCE'] = 'HISTO ARO'

HISTO_WOH_BIC = pd.read_csv(path_in + '/HISTORICAL_WO_HEADER_BIC.txt',delimiter='|',
                           dtype={'PNM':'str','WONO':'str'})
HISTO_WOH_BIC = HISTO_WOH_BIC.drop(columns=['ID'])
HISTO_WOH_BIC['ENTRY_DATE'] = pd.to_datetime(HISTO_WOH_BIC['ENTRY_DATE'],  format='%m/%d/%Y %H:%M:%S')
HISTO_WOH_BIC['CLOSE_DATE'] = pd.to_datetime(HISTO_WOH_BIC['CLOSE_DATE'],  format='%m/%d/%Y %H:%M:%S')
HISTO_WOH_BIC['DATA_SOURCE'] = 'HISTO BIC'

#append histoical data to current data
WO_HEADER= WO_HEADER.append(HISTO_WOH_ARO, ignore_index = True, sort = False)
WO_HEADER= WO_HEADER.append(HISTO_WOH_BIC, ignore_index = True, sort = False)

#clean data
WO_HEADER = WO_HEADER.loc[WO_HEADER['GEO_CODE'] != 'TEST']
WO_HEADER = WO_HEADER.loc[~WO_HEADER['GEO_CODE'].isnull()]

#save as pickle
WO_HEADER.to_pickle(path_out + '/WO_HEADER.pkl')


## Capabilities data

In [121]:
CAPABILITIES = pd.read_csv(path_in + '/ba_view_internal_capabilities.txt',delimiter='|')
CAPABILITIES['IS_ACTIVE'] = np.where(CAPABILITIES['CAPABILITY_CODE'].str.contains('INACTIVE'),
                                     '0', '1') #adds colunm to show active caps

#CAPABILITIES
CAPABILITIES = CAPABILITIES[CAPABILITIES['CAPABILITY_CODE'] != 'Barfield, Inc.']
CAPABILITIES = CAPABILITIES[~CAPABILITIES['CAPABILITY_CODE'].isnull()]

#save as pickle
CAPABILITIES.to_pickle(path_out + '/CAPABILITIES.pkl')

## All other Static files (not sure that this is useful)

In [2]:
DISTRIB_PARTS = pd.read_csv(path_in + '/BICdistributedparts.csv',delimiter=';',
                                dtype={
                                    'PNM':'str',
                                    'PN':'str'
                                })
EXCLUD_PARTS_SO = pd.read_csv(path_in + '/ExcludedPartsForSalesOrders.csv',
                              delimiter=',',
                              dtype={
                                     'PN':'str' 
                              })
EXCLUD_CLIENT_SO = pd.read_csv(path_in + '/ExcludedClientsForSalesOrders.csv',
                               delimiter=',')
EXCLUD_SO = pd.read_csv(path_in + '/ExcludedSalesOrders.csv',delimiter=',')
EXCLUD_PARTS = pd.read_csv(path_in + '/PartsToBeExcluded.csv',delimiter=',')
MANUAL_MIN_MAX = pd.read_csv(path_in + '/min_max_static.csv',delimiter=',',
                            dtype={
                                'PN':'str'
                            })
PMA_ACCEPTANCE = pd.read_csv(path_in + '/pma_acceptance.csv',delimiter=',',
                            dtype={'PN':'str'})
VIP_LRU = pd.read_csv(path_in + '/VIP_LRUs.csv',delimiter=',')
WHS = pd.read_csv(path_in + '/WHS.csv',delimiter=',')
WON_BIZ = pd.read_csv(path_in + '/WON_BUSINESS.csv',delimiter=',')
LOST_BIZ = pd.read_csv(path_in + '/LOST_BUSINESS.csv',delimiter=',')